In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import math

import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs_physicsVars import Mbc_Btag, MM2recoilSignalSide, deltaE

In [2]:
path_df = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/MM2plotFromPreds/"

In [3]:
df = pd.read_csv(path_df + "Gmodes_fixedD0_df_preds_onTestSet.csv")

In [5]:
numFSPs = pd.DataFrame({'numFSPs' : df.groupby( ["event"] ).size()}).reset_index()

In [7]:
percCorrect_df = pd.DataFrame({'numErrors' : df.groupby(["event","correct"] ).size()}).reset_index()
percCorrect_df = percCorrect_df[percCorrect_df["correct"] == 0]
percCorrect_df

,event,correct,numErrors
0,36588,0,4
2,37464,0,1
4,37653,0,6
6,38101,0,3
8,38479,0,1
...,...,...,...
100772,50998090,0,2
100774,50998482,0,5
100777,50999524,0,6
100779,50999934,0,4


In [8]:
NN_results = pd.merge(percCorrect_df[["event","numErrors"]],numFSPs, on="event",how="outer")


In [9]:
NN_results[NN_results["numErrors"].isna()].shape[0]

6949

In [10]:
NN_results['numErrors'] = NN_results['numErrors'].fillna(0)

In [11]:
NN_results[NN_results["numErrors"].isna()].shape[0]

0

In [12]:
NN_results["numErrors"].value_counts(normalize=True)

2.0     0.169866
1.0     0.166227
3.0     0.152046
0.0     0.128991
4.0     0.128471
5.0     0.097824
6.0     0.066510
7.0     0.043622
8.0     0.023574
9.0     0.012474
10.0    0.005587
11.0    0.002822
12.0    0.001188
13.0    0.000483
14.0    0.000223
15.0    0.000093
Name: numErrors, dtype: float64

In [18]:
type(NN_results["numErrors"].value_counts())

pandas.core.series.Series

In [29]:
valueCounts = (NN_results["numFSPs"].value_counts())
valueCounts = valueCounts.sort_index()
valueCounts

5      569
6     1234
7     2060
8     2871
9     3729
10    4411
11    5019
12    5378
13    5401
14    5162
15    4688
16    3929
17    3069
18    2329
19    1626
20    1071
21     677
22     410
23     239
Name: numFSPs, dtype: int64

In [35]:
indices = valueCounts.index
numFSPs=[]
for i in range(len(valueCounts)):
    #print(indices[i],valueCounts[indices[i]])
    numFSPs.append(valueCounts[indices[i]])

In [66]:
errors_list=[0,0,0,0,0]
totalSamples=0

for i in range(len(valueCounts)):
    n = indices[i]
    p = 0.768
    n_samples = numFSPs[i]*100
    totalSamples+=n_samples
    sample = np.random.binomial(n, p,(n_samples,1))


    tmp = np.full((n_samples,1), n)
    errors = tmp - sample

    for i in range(4):
        num_errors = (errors == i).sum()
        errors_list[i] += num_errors
        #print(i,"errors:",round(num_errors/n_samples*100,2),"% of events")

    num_errors = (errors > 3).sum()
    errors_list[4] += num_errors
    
print("p:",p,"result:",(errors_list/totalSamples))    


p: 0.768 result: [0.05139052 0.15190786 0.22306449 0.22092441 0.35271273]


In [62]:
sum(errors_list) == totalSamples

True

In [63]:
errors_list/totalSamples

array([0.0512719 , 0.15167731, 0.22314709, 0.22103987, 0.35286383])

In [37]:
ErrorNNdistri = (NN_results["numErrors"].value_counts(normalize=True)).sort_index()
ErrorNNdistri

0.0     0.128991
1.0     0.166227
2.0     0.169866
3.0     0.152046
4.0     0.128471
5.0     0.097824
6.0     0.066510
7.0     0.043622
8.0     0.023574
9.0     0.012474
10.0    0.005587
11.0    0.002822
12.0    0.001188
13.0    0.000483
14.0    0.000223
15.0    0.000093
Name: numErrors, dtype: float64

In [67]:
root_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/BASF2_performance_GiannasScript/GiannasScript_wNNcuts_D0fixed/"

In [68]:
df_FSPs = pd.read_csv(root_path + "FSPs_preProcessed_basf2Perf.csv")

In [70]:
df_FSPs["event"] = df_FSPs["__event__"]

In [71]:
numFSPs = pd.DataFrame({'numFSPs' : df_FSPs.groupby( ["event"] ).size()}).reset_index()

In [72]:
percCorrect_df = pd.DataFrame({'numErrors' : df_FSPs.groupby(["event","basf2_correct"] ).size()}).reset_index()
percCorrect_df = percCorrect_df[percCorrect_df["basf2_correct"] == 0]
percCorrect_df

,event,basf2_correct,numErrors
0,579,False,11
2,56345,False,7
4,59093,False,3
6,61733,False,3
8,134598,False,3
...,...,...,...
3452,50819999,False,2
3454,50839237,False,2
3456,50845012,False,2
3458,50864508,False,2


In [73]:
NN_results = pd.merge(percCorrect_df[["event","numErrors"]],numFSPs, on="event",how="outer")


In [74]:
NN_results[NN_results["numErrors"].isna()].shape[0]

62

In [75]:
NN_results['numErrors'] = NN_results['numErrors'].fillna(0)

In [76]:
NN_results[NN_results["numErrors"].isna()].shape[0]

0

In [78]:
NN_results["numErrors"].value_counts(normalize=True).sort_index()

0.0     0.035187
1.0     0.092509
2.0     0.150397
3.0     0.148127
4.0     0.129966
5.0     0.124858
6.0     0.094211
7.0     0.070942
8.0     0.050511
9.0     0.036890
10.0    0.030647
11.0    0.017026
12.0    0.010783
13.0    0.002838
14.0    0.002838
15.0    0.001703
17.0    0.000568
Name: numErrors, dtype: float64

In [79]:
valueCounts = (NN_results["numFSPs"].value_counts())
valueCounts = valueCounts.sort_index()
valueCounts

5       1
6       3
7       7
8      29
9      35
10     66
11     87
12    122
13    134
14    167
15    147
16    159
17    174
18    128
19    131
20    113
21     91
22     56
23     56
24     27
25     14
26     15
Name: numFSPs, dtype: int64

In [80]:
indices = valueCounts.index
numFSPs=[]
for i in range(len(valueCounts)):
    #print(indices[i],valueCounts[indices[i]])
    numFSPs.append(valueCounts[indices[i]])

In [82]:
errors_list=[0,0,0,0,0]
totalSamples=0

for i in range(len(valueCounts)):
    n = indices[i]
    p = 0.72
    n_samples = numFSPs[i]*10000
    totalSamples+=n_samples
    sample = np.random.binomial(n, p,(n_samples,1))


    tmp = np.full((n_samples,1), n)
    errors = tmp - sample

    for i in range(4):
        num_errors = (errors == i).sum()
        errors_list[i] += num_errors
        #print(i,"errors:",round(num_errors/n_samples*100,2),"% of events")

    num_errors = (errors > 3).sum()
    errors_list[4] += num_errors
    
print("p:",p,"result:",(errors_list/totalSamples))    


p: 0.72 result: [0.01103899 0.05006771 0.1125252  0.16808785 0.65828025]
